# Urban Head Index

By Cascade Tuholske 2020.01.21

Notebook is designed to take areal-averaged CHIRTS Tmax for each GHS-UCDB and down-scaled MERRA-2 humidity data and calculate the heat index for each city with a Tmax >80F.

[NOAA Heat Index Equation](https://www.wpc.ncep.noaa.gov/html/heatindex_equation.shtml)

**THINGS I WILL NEED TO CODE**
- C to F function
- Rothfusz regression and adjustments
- Steadman's

#### Dependencies

In [2]:
import pandas as pd
import numpy as np
import xarray as xr
from random import random
from itertools import groupby
from operator import itemgetter
import geopandas as gpd 
import glob
from statistics import mean
import julian
import math

## Functions

In [3]:
def C_to_F(Tmax_C):
    "Function converts temp in c to f"
    Tmax_F = (Tmax_C * (9/5)) + 32
    
    return Tmax_F

In [4]:
def steadman_hi(Tmax_F, RH):
    "Simple heat index calculation"
    
    HI_steadman = 0.5 * (Tmax_F + 61.0 + ((Tmax_F-68.0)*1.2) + (RH*0.094))
    
    HI_steadman = (HI_steadman + Tmax_F) / 2
    
    return HI_steadman

In [5]:
def rothfusz_hi(HI_steadman, Tmax_F, RH):
    
    "Heat Index applied to Steadman's heat index >80F"
    
    #if HI_steadman > 80:
    
        HI_rothfusz = -42.379 + 2.04901523*Tmax_F + 10.14333127*RH - .22475541*Tmax_F*RH - .00683783*Tmax_F*Tmax_F - .05481717*RH*RH + .00122874*Tmax_F*Tmax_F*RH + .00085282*Tmax_F*RH*RH - .00000199*Tmax_F*Tmax_F*RH*RH
    
    if RH < 13 and Tmax_F > 80 and Tmax_F < 112:
        adjustment = ((13-RH)/4)*math.sqrt((17-abs(Tmax_F-95))/17)
        HI_rothfusz = HI_rothfusz - adjustment 
    
    "If the RH is greater than 85% and the temperature is between 80 and 87 degrees F" 
    if RH > 85 and Tmax_F > 80 and Tmax_F < 87:
        adjustment = ((RH-85)/10) * ((87-Tmax_F)/5)
        HI_rothfusz = HI_rothfusz + adjustment 
    
    return HI_rothfusz

IndentationError: unexpected indent (<ipython-input-5-a415c133adac>, line 7)

In [6]:
def make_hi(Tmax_F, RH):
    "Calculates the heat index for a CHIRTS Tmax value"
    Tmax_F = C_to_F(Tmax_C)
#     print('F is ', Tmax_F)
#     print('H is ', RH)
    HI_steadman = steadman_hi(Tmax_F, RH)
#     print('HI_steadman is ', HI_steadman)
    HI_rothfusz = rothfusz_hi(HI_steadman, Tmax_F, RH)
#     print('HI_rothfusz is ', HI_rothfusz)
    
    return HI_rothfusz

#### Test Heat Index

Get some paired values from the [NOAA CHART](https://www.weather.gov/safety/heat-index) and test the output. 

These results seem to be working well, though 104 F and 60 RH produced a higher number than the chart ...

In [ ]:
temp = [25, 27, 35, 40, 41] # temp C
RH = 60
test = pd.DataFrame()
test['temp'] = temp
test['RH'] = RH

In [ ]:
for i, row in test.iterrows():
    out = make_hi(row['temp'], row['RH'])
    print('\n')

# Next we need to read in the csv files and quickly calculate the HI

In [7]:
DIR_RH = '/home/cascade/projects/UrbanHeat/data/interim/RH-GHS-DIALY/'
DIR_RAW = '/home/cascade/projects/UrbanHeat/data/interim/CHIRTS-GHS-DAILY-RAW/'

In [8]:
FN_RH = 'GHS-Tmax-RH_1983.csv'
FN_RAW = 'GHS-Tmax-DAILY_1983.csv'

In [9]:
RH = pd.read_csv(DIR_RH+FN_RH)
RAW = pd.read_csv(DIR_RAW+FN_RAW)

In [10]:
df_temp_id = RAW['ID_HDC_G0'] # get IDs
df_temp = RAW.iloc[:,3:] # get only temp columns
df_temp.index = df_temp_id # set index values
df_temp_drop = df_temp.dropna() # Drop cities w/ no temp record 
print(len(df_temp_drop))

13067


In [11]:
df_temp_drop.head()

,1983.01.01,1983.01.02,1983.01.03,1983.01.04,1983.01.05,1983.01.06,1983.01.07,1983.01.08,1983.01.09,1983.01.10,...,1983.12.22,1983.12.23,1983.12.24,1983.12.25,1983.12.26,1983.12.27,1983.12.28,1983.12.29,1983.12.30,1983.12.31
ID_HDC_G0,,,,,,,,,,,,,,,,,,,,,
5782,-43.921947,-33.713450,-33.054974,-25.376320,-23.129280,-21.466286,-19.981667,-15.222391,-16.233010,-16.017557,...,-24.950730,-35.233840,-26.349930,-17.329056,-12.715102,-12.801040,-15.181746,-12.416152,-13.232986,-15.403823
3316,-4.804248,-3.914425,-7.533999,-12.615092,-7.848448,-6.586810,-5.604166,-5.210299,-7.793556,-5.328217,...,-11.624672,-17.263590,-6.157263,-7.405956,-11.043344,-15.030403,-13.661594,-5.186461,-10.945722,-16.295160
5645,-23.904118,-17.422953,-13.182008,-14.080582,-15.031302,-17.476360,-21.271954,-12.573436,-7.619316,-9.144251,...,-20.022259,-21.562733,-12.196519,-3.379593,-5.561025,-8.924066,-11.111601,-12.788978,-11.337886,-10.009390
3185,-0.145468,-0.141592,-4.062948,-2.354636,-1.533712,-0.494162,0.049387,0.149454,-1.632070,-4.892538,...,-9.276704,-3.720002,-3.007447,-8.143495,-9.042634,-3.481507,-3.199769,-6.058533,-9.540711,-8.047456
3539,-6.093382,-6.081639,-15.774755,-13.649839,-9.780418,-2.830632,1.321785,1.234120,-0.136949,-3.587718,...,-4.097543,-1.184109,-0.788187,-0.913870,-6.033908,-8.862185,-8.945712,-7.596188,-6.915255,-9.641947


In [12]:
df_RH_id = RH['ID_HDC_G0'] # get IDs
df_RH = RH.iloc[:,3:] # get only temp columns
df_RH.index = df_RH_id # set index values
df_RH_drop = df_RH.dropna() # Drop cities w/ no temp record 
print(len(df_RH_drop))

13067


In [13]:
# Make them into a data array

temp_xr_da = xr.DataArray(df_temp_drop, coords=[df_temp_drop.index, df_temp_drop.columns], 
                            dims=['ID_HDC_G0', 'date'])
RH_xr_da = xr.DataArray(df_RH_drop, coords=[df_RH_drop.index, df_RH_drop.columns], 
                            dims=['ID_HDC_G0', 'date'])

In [14]:
# turn the temp x array from C to F

temp_xr_da = C_to_F(temp_xr_da)

In [15]:
temp_xr_da

<xarray.DataArray (ID_HDC_G0: 13067, date: 365)>
array([[-47.059505, -28.68421 , -27.498953, ...,   9.650926,   8.180626,
          4.273119],
       [ 23.352354,  24.954035,  18.438803, ...,  22.664369,  12.2977  ,
          2.668712],
       [-11.027412,   0.638685,   8.272386, ...,   8.97984 ,  11.591805,
         13.983098],
       ...,
       [ 60.850441,  63.924854,  68.887929, ...,  58.206958,  59.28933 ,
         59.331243],
       [ 61.556995,  64.168556,  72.535413, ...,  60.225735,  61.105519,
         60.870522],
       [ 62.04974 ,  63.606612,  70.665654, ...,  58.002894,  59.424084,
         57.210064]])
Coordinates:
  * ID_HDC_G0  (ID_HDC_G0) int64 5782 3316 5645 3185 ... 1116 1114 1161 1169
  * date       (date) object '1983.01.01' '1983.01.02' ... '1983.12.31'

In [16]:
# apply steadman's

HI_steadman = steadman_hi(temp_xr_da, RH_xr_da)

In [17]:
HI_steadman

<xarray.DataArray (ID_HDC_G0: 13067, date: 365)>
array([[-52.79085 , -33.209534, -32.424601, ...,   7.333473,   5.723379,
          1.411044],
       [ 21.612108,  23.401737,  16.560743, ...,  20.97596 ,   9.666459,
         -0.359718],
       [-15.149145,  -2.844831,   5.225676, ...,   5.991969,   8.79311 ,
         11.260339],
       ...,
       [ 60.131937,  63.263469,  68.54139 , ...,  57.353674,  58.46652 ,
         58.646119],
       [ 60.943873,  63.562727,  72.311917, ...,  59.511969,  60.38539 ,
         60.219069],
       [ 61.211956,  62.817895,  70.245043, ...,  57.073305,  58.541903,
         56.282076]])
Coordinates:
  * ID_HDC_G0  (ID_HDC_G0) int64 5782 3316 5645 3185 ... 1116 1114 1161 1169
  * date       (date) object '1983.01.01' '1983.01.02' ... '1983.12.31'

In [55]:
def hi(Tmax_F, RH):
    
    "Heat Index applied to Steadman's heat index >80F"
    
    USE_STEAD = (0.5 * (Tmax_F + 61.0 + ((Tmax_F-68.0)*1.2) + (RH*0.094)) + Tmax_F)/2 < 80  
    STEAD = 0.5 * (Tmax_F + 61.0 + ((Tmax_F-68.0)*1.2) + (RH*0.094)) * USE_STEAD.astype(int)
    
    ROTHFUZ = -42.379 + 2.04901523*Tmax_F + 10.14333127*RH - .22475541*Tmax_F*RH - .00683783*Tmax_F*Tmax_F - .05481717*RH*RH + .00122874*Tmax_F*Tmax_F*RH + .00085282*Tmax_F*RH*RH - .00000199*Tmax_F*Tmax_F*RH*RH
    ADJ1 = ((13-RH)/4)*np.sqrt((17-abs(Tmax_F-95))/17)
    ADJ2 = ((RH-85)/10) * ((87-Tmax_F)/5)
    ADJ = ADJ1*((Tmax_F < 112) & (RH < 13)).astype(int) + ADJ2*((Tmax_F < 87) & (RH > 85)).astype(int)
    ANS = STEAD + (ROTHFUZ + ADJ) * np.logical_not(USE_STEAD).astype(int)
    return ANS
#    HI_steadman = 
#     if RH < 13 and Tmax_F > 80 and Tmax_F < 112:
#         adjustment = ((13-RH)/4)*math.sqrt((17-abs(Tmax_F-95))/17)
#         HI_rothfusz = HI_rothfusz - adjustment 
    
#     "If the RH is greater than 85% and the temperature is between 80 and 87 degrees F" 
#     if RH > 85 and Tmax_F > 80 and Tmax_F < 87:
#         adjustment = ((RH-85)/10) * ((87-Tmax_F)/5)
#         HI_rothfusz = HI_rothfusz + adjustment 
#    
#    return HI_steadman

In [56]:
hi(Tmax_F, RH)

/home/cascade/miniconda3/envs/geo/lib/python3.6/site-packages/xarray/core/computation.py:564: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


<xarray.DataArray (ID_HDC_G0: 13067, date: 365)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * ID_HDC_G0  (ID_HDC_G0) int64 5782 3316 5645 3185 ... 1116 1114 1161 1169
  * date       (date) object '1983.01.01' '1983.01.02' ... '1983.12.31'

In [31]:
Tmax_F = temp_xr_da
RH = RH_xr_da
USE_STEAD = (0.5 * (Tmax_F + 61.0 + ((Tmax_F-68.0)*1.2) + (RH*0.094)) + Tmax_F)/2 < 80


In [36]:
dir(USE_STEAD)

['ID_HDC_G0',
 'T',
 '_DataArray__default',
 '_DataArray__default_name',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__dask_graph__',
 '__dask_keys__',
 '__dask_layers__',
 '__dask_optimize__',
 '__dask_postcompute__',
 '__dask_postpersist__',
 '__dask_scheduler__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',


In [27]:
np.logical_not(a)

array([False,  True,  True])

In [ ]:
def csv_to_xr(file_in, time_dim, space_dim):
    
    """ Function reads in a csv w/ GHS-UCDB IDs and temp, isolates the temp
    and returns a xarray data array with dims set to city ids and dates
    
    Args:
        file_in = file name and path
        time_dim = name for time dim as a str ... use date :-)
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0)
    """
    
    df = pd.read_csv(file_in) # read the file in as a df
    print(df.shape)
    
    df_id = df[space_dim] # get IDs
    df_temp = df.iloc[:,3:] # get only temp columns
    df_temp.index = df_id # set index values
    df_temp_drop = df_temp.dropna() # Drop cities w/ no temp record 
    print(len(df_temp_drop))
    
    temp_np = df_temp_drop.to_numpy() # turn temp cols into an np array
    
    # make xr Data Array w/ data as temp and dims as spece (e.g. id)
    
    # Note 2019 09 17 changed to xr.Dataset from xr.Dataarray
    temp_xr_da = xr.DataArray(temp_np, coords=[df_temp_drop.index, df_temp_drop.columns], 
                            dims=[space_dim, time_dim])
    
    return temp_xr_da

#### #2 Function finds all the Tmax Events and writes it to a dateframe w/ dates for each city

In [ ]:
def tmax_days(xarray, Tthresh):
    """ Function finds all the tmax days in a year and sums total days per year 
    greater than a threshold within a year where Tmax > Tthresh for each city. Returns the total number of days,
    the dates, the tempatures, and the intensity (daily Tmax - Tthresh)
    
    Args: 
        xarray = an xarray object with dims = (space, times)
        Tthresh = int of temp threshold
    """
    
    # empty lists & df
    id_list = []
    date_list = []
    dayTot_list = []
    tmax_list = []
    intensity_list = []
    df_out = pd.DataFrame()
    
    # subset xarray
    out = xarray.where(xarray > Tthresh, drop = True)

    # start loop 
    for index, loc in enumerate(out.ID_HDC_G0):
        id_list.append(out.ID_HDC_G0.values[index]) # get IDS
        date_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values) # get event dates
        
        # this is actually getting the total events of all, 2019-09-22
        dayTot_list.append(len(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').date.values)) # get event totals
        
        tmax_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values) # get temp values
        intensity_list.append(out.sel(ID_HDC_G0 = loc).dropna(dim = 'date').values - Tthresh) # get severity

    # write to a data frame
    df_out['ID_HDC_G0'] = id_list
    df_out['total_days'] = dayTot_list
    df_out['dates'] = date_list
    df_out['tmax'] = tmax_list
    df_out['tmax_tntensity'] = intensity_list

    # return df_out
    return df_out

#### #3 Function splits the dataset into Tmax events (continuous days >Tmax) for each city

In [ ]:
def jul_convert(dates):
    "Function turn days into julian datetime"
    jul_days = pd.to_datetime(dates).to_julian_date()
    
    return jul_days

def event_split(dates, ID_HDC_G0, intensity, tmax, total_days):
    """ Searchs a list of dates and isolates sequential dates as a list, then calculates event stats.
    See comments in code for more details. 
    
    Args:
        dates: pandas.core.index as julian dates
        ID_HDC_G0: city ID as string
        country: country for each city as string
        intensity: numpy.ndarray of intensities values
        tmax: numpy.ndarray of intensities values of tmax values
        total_days: total number of tmax days in a year for a given city

    """

    # city id
    city_id = ID_HDC_G0
    tot_days = total_days
    
    # lists to fill
    city_id_list = []
    tot_days_list = []
    event_dates_list = []
    dur_list = []
    intensity_list = []
    tmax_list = []
    avg_temp_list = []
    avg_int_list = []
    tot_int_list = []
    
    # data frame out
    df_out = pd.DataFrame()
    
    # turn days into julian days
    jul_days = jul_convert(dates)
    
    # Counters to make sure we write the correct event dates to a list, don't want julian days in output
    counter = 0
    start = 0
    end = 0
    
    # Loop through dur list and isolate seq days, temps, and intensities
    for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
        
        seq = list(map(itemgetter(1), g)) # isolate seq. days
        dur = len(seq) # duration of each event
        
        counter = counter + dur # add duration to counter
        end = counter # end of current event
        
        event_dates = dates[start:end] # dates of tmax days during each event
        intense = intensity[start:end] # intensity of each day during event
        temp = tmax[start:end] # temp of each day during event
        avg_temp = mean(temp) # avg. temp during event
        avg_int = mean(intense) # avg. intensity during event
        tot_int = intense.sum() # total intensity during event
        
        start = counter # reset start to current end (e.g. counter)
        
        # fill lists
        city_id_list.append(city_id)
        tot_days_list.append(tot_days)
        dur_list.append(dur)
        event_dates_list.append(event_dates)
        intensity_list.append(intense)
        tmax_list.append(temp)
        avg_temp_list.append(avg_temp)
        avg_int_list.append(avg_int)
        tot_int_list.append(tot_int)

    # write out as a dateframe
    df_out['ID_HDC_G0'] = city_id_list
    df_out['total_days'] = tot_days_list
    df_out['duration'] = dur_list
    df_out['avg_temp'] = avg_temp_list
    df_out['avg_intensity'] = avg_int_list
    df_out['tot_intensity'] = tot_int_list
    df_out['event_dates'] = event_dates_list
    df_out['duration'] = dur_list
    df_out['intensity'] = intensity_list
    df_out['tmax'] = tmax_list

    return df_out

#### #4 Function feeds output from function 2 into function 3

In [ ]:
def tmax_stats(df_in):
    """ runs event_split functionon a dataframe to produce desired tmax stats
    
        NOTE - If you add arguments to event_split to make more states, 
        be sure to update this function
    
        args:
            df: input dataframe
        
    """
    df_out = pd.DataFrame()
    
    # NOTE - If you add arguments to event_split to make more stats, 
    # be sure to update this function
    
    for index, row in df_in.iterrows():
        dates = row['dates'] # Get event dates
        intensity = row['tmax_tntensity'] # Get intensity for each day
        tmax = row['tmax'] # Get tmax for each day
        ID_HDC_G0 = row['ID_HDC_G0'] # get city id
        total_days = row['total_days'] # get total number of tmax days

        df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)

        df_out = df_out.append(df)
    
    return df_out

#### #5 Loops through a file list and applies functions 1 - 4 to the data to produce Tmax stats for all tmax events in a given year across all cities in the dataset

In [ ]:
def stats_loop(dir_in, dir_out, fn_out, time_dim, space_dim, Tthresh):
    
    """ Loop through a dir with csvs to apply csv_to_xr and
    tmax_stats function and save out a .csv for each year
    
    Args:
        dir_in = dir path to loop through
        dir_out = dir path to save files out
        fn_out = string to label out files
        time_dim = name for time dim as a str ... use date :-) for csv_to_xr function
        space_dim = col name for GHS-UCDB IDs as an str (ID_HDC_G0) for csv_to_xr function
        Tthresh = int of temp threshold for temp_event function -- 40.6 is used

    """
    
    # Open the GHS-ID List with GeoPANDAS read_file
    ghs_ids_fn = 'GHS-UCSB-IDS.csv'
    ghs_ids_df = pd.read_csv(DATA_INTERIM+ghs_ids_fn)
        
    # Git File list
    fn_list = glob.glob(dir_in+'*.csv')
    
    for fn in sorted(fn_list):
        
        # Get year for arg for temp_event function
        year = fn.split('GHS-Tmax-DAILY_')[1].split('.csv')[0]
        print(year)
        
        # read csv as a data array
        temp_xr_da = csv_to_xr(fn, time_dim, space_dim)
        
        # data array to tmax events, out as df
        df_days = tmax_days(temp_xr_da, Tthresh)
        
        # tmax events stats, out as df
        df_out = tmax_stats(df_days)
        
        # merge to get countries
        ghs_ids_df_out = ghs_ids_df.merge(df_out, on='ID_HDC_G0', how = 'inner') 
        
        # write it all out
        ghs_ids_df_out.to_csv(dir_out+fn_out+year+'.csv')

        print(year, 'SAVED!')

#### Run Code

In [ ]:
dir_in = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/UrbanHeat/data/interim/' # ghs ID list
dir_out = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-Events-Stats/'
fn_out = 'CHIRTS-GHS-Events-Stats'
time_dim = 'date'
space_dim = 'ID_HDC_G0'
Tthresh = 40.6


In [ ]:
#stats_loop(dir_in, dir_out, fn_out, time_dim, space_dim, Tthresh)

## QA/QC

In [ ]:
# Load files

dir_out = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-Events-Stats/'
fn_out = 'CHIRTS-GHS-Events-Stats'

In [ ]:
qc_df = pd.read_csv(dir_out+fn_out+'1983.csv')

In [ ]:
qc_df.head()

In [ ]:
dc_city = qc_df[qc_df['ID_HDC_G0'] == 5534]

In [ ]:
dc_city

## Testing

In [ ]:
# File Paths
DAILY_PATH = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/data_out_urbanheat/data/interim/'
DATA_OUT = '/home/cascade/projects/data_out/'

In [ ]:
# File name to test
fn_in = 'GHS-Tmax-DAILY_1983.csv'

In [ ]:
# Open a raw file
xr1983 = csv_to_xr(DAILY_PATH+fn_in, 'date', 'ID_HDC_G0')

In [ ]:
days1983 = tmax_days(xr1983, 40.6)

In [ ]:
test = days1983[0:30]
test

# Maybe add in days_total

In [ ]:
# Build routine for loop through a csv

df_out = pd.DataFrame()

for index, row in test.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days
    
    df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)
    
    df_out = df_out.append(df)

In [ ]:
df_out

# Fixing code
2019.10.19 Cascade Tuholkse

Need to fix ```event split``` function

Somewhere in 1984 is this event sequence: ['1984.01.01' '1984.01.02' '1984.01.07']


**FOUND PROBLEM AND IT IS FIXED**

In [ ]:
# File Paths
DAILY_PATH = '/home/cascade/projects/data_out_urbanheat/CHIRTS-GHS-DAILY/' # output from avg temp
DATA_INTERIM = '/home/cascade/projects/data_out_urbanheatv/data/interim/'
DATA_OUT = '/home/cascade/projects/data_out_urbanheat/testout/'

In [ ]:
# File name to test
fn_in = 'GHS-Tmax-DAILY_1983.csv'

In [ ]:
# Open a raw file
xr1983 = csv_to_xr(DAILY_PATH+fn_in, 'date', 'ID_HDC_G0')

In [ ]:
# Find all the tmax days
tmax1983 = tmax_days(xr1983, 40.6)

In [ ]:
# Make a subset

test = tmax1983[tmax1983['ID_HDC_G0'] == 6279]
test

In [ ]:
for index, row in test.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days



In [ ]:
# Convert to jul days
jul_days = pd.to_datetime(dates).to_julian_date()
jul_days


In [ ]:
mjd = 2445507.5 - 43200
dt = julian.from_jd(mjd, fmt='mjd')
print(dt)


In [ ]:
dates

In [ ]:
dates = ['1983.06.20', '1983.06.23', '1983.06.24', '1983.06.25',
        '1983.06.26', '1983.06.27', '1983.06.28', '1983.06.29',
        '1983.06.30', '1983.07.01', '1983.07.21', '1983.07.22',
        '1983.07.23', '1983.08.01']

pd_dates = pd.to_datetime(dates)
df_dates = pd.DataFrame()
df_dates['dates'] = pd_dates



test = df_dates['dates'].apply(lambda x: x.toordinal())

In [ ]:
from datetime import date
date.fromordinal(test[1])

In [ ]:
counter = 0

In [ ]:
event_dates = dates[0:10] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

event_dates = dates[10:13] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

event_dates = dates[13:14] # dates of tmax days during each event
print(event_dates)
print(len(event_dates))

In [ ]:
event_dates_list = []
counter = 0
start = 0
end = 0

for k, g in groupby(enumerate(jul_days.values), lambda x: x[1]-x[0]):
    
    len_dates = len(dates) # len of all Tmax dates for a given city
#   print(len(dates))
    
    seq = list(map(itemgetter(1), g)) # isolate seq. days
    dur = len(seq) # duration of each event 
    
    counter = counter + dur
    end = counter
    
    print(end)
    
    event_dates = dates[start:end] # dates of tmax days during each event
    print(event_dates)
    
    start = counter
    
# #     print('dur= ', dur)
    
#     event_dates = dates[0:10] # dates of tmax days during each event
#     print(event_dates)
    
#     event_dates = dates[11:13] # dates of tmax days during each event
#     print(event_dates)
    
#     event_dates = dates[:len_dates] # dates of tmax days during each event
#     print(event_dates)
    
    
    
    
#     counter = counter + dur

#     print('counter = ', counter)
    
#     dif = dur - counter
#     print('dif = ', dif)
    
#     start = start 
#     print('start = ',start)
    
# #     end = counter + dur
# #     print("start = ",end)
    
    
    
#     counter = counter + dur
#     print(counter)
#     end = counter + dur
#     start
#     event_dates = dates[start:end] # dates of tmax days during each event
#     print(event_dates)
# #     intense = intensity[0:dur] # intensity of each day during event
#     temp = tmax[0:dur] # temp of each day during event
#     avg_temp = mean(temp) # avg. temp during event
#     avg_int = mean(intense) # avg. intensity during event
#     tot_int = intense.sum() # total intensity during event
    
#     event_dates_list.append(event_dates)
    


In [ ]:
event_dates_list

In [ ]:
df = pd.DataFrame()
df['event_dates'] = event_dates_list
df

In [ ]:
import more_itertools as mit

In [ ]:
# run tmax stats ------> CLEARLY NOTE WORKING

# tmax1983_sub_stats = tmax_stats(tmax1983_sub)

In [ ]:
type(tmax1983['dates'][1])

In [ ]:
dates = tmax1983_sub['dates']
#jul_days = pd.to_datetime(tmax1983['dates']).to_julian_date()

In [ ]:
# Try to break up events

for index, row in tmax1983_sub.iterrows():
    dates = row['dates'] # Get event dates
    intensity = row['tmax_tntensity'] # Get intensity for each day
    tmax = row['tmax'] # Get tmax for each day
    ID_HDC_G0 = row['ID_HDC_G0'] # get city id
    total_days = row['total_days'] # get total number of tmax days

#     df = event_split(dates, ID_HDC_G0, intensity, tmax, total_days)

#     df_out = df_out.append(df)

dates

In [ ]:
type(df_out['events'][0])